In [ ]:
# # use erddapy to get some data
# from erddapy import ERDDAP
# import numpy as np
# 
# e = ERDDAP(
#     server="https://erddap.marine.usf.edu/erddap",
#     protocol="griddap",
# )
# 
# e.dataset_id = "jplMURSST41anom1day"
# print(f"variables in this dataset:\n\n{e.variables}")
# 
# # set bounds to reduce download size
# e.constraints = {
#     'time>=': '2003-06-07T09:00:00Z', 
#     'time<=': '2024-06-07T09:00:00Z', 
#     'time_step': 1,
#     'latitude>=': np.float64(lat - 1),
#     'latitude<=': np.float64(lat + 1),
#     'latitude_step': 1,
#     'longitude>=': np.float64(lon - 1),
#     'longitude<=': np.float64(lon + 1),
#     'longitude_step': 1
# }
# 
# Download as a NetCDF to disk
# nc_filename = e.download_file(".nc")  
# (under the hood this uses the GRiDDAP `.nc` URL)
# 
# ds = e.to_xarray()

In [ ]:
# sample all taxa occurrence points for ERDDAP data values:
# data has already been downloaded as .nc
# example:
# https://erddap.marine.usf.edu/erddap/griddap/cmems_salinity.graph?so%5B(2023-06-16)%5D%5B(0.494025)%5D%5B(-80.0):(90.0)%5D%5B(-180.0):(179.9167)%5D&.draw=surface&.vars=longitude%7Clatitude%7Cso&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff

import xarray as xr
import pandas as pd

# load taxa occurrence points
occurrences = pd.read_csv("data/occurrences.csv")


# Open raster data file with xarray
ds = xr.open_dataset("data/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc")

# print(occurrences["decimalLatitude"][0])

# add a column to occurrences for the salinity value
occurrences["salinity"] = ds["so"].sel(
    latitude=occurrences["decimalLatitude"],
    longitude=occurrences["decimalLongitude"],
    method="nearest"
).values





# for occurrence in occurrences.iterrows():
#     # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
#     print(occurrence["decimalLatitude"])
#     val = ds["so"].sel(
#         latitude=occurrence["decimalLatitude"],
#         longitude=occurrence["decimalLongitude"],
#         method="nearest"
#     ).values

#     print(f"At ({occurrence["decimalLatitude"]}, {occurrence["decimalLongitude"]}), so = {val}")

# lat = 60
# lon = -60

# # Open file with xarray
# ds = xr.open_dataset("data/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc")


# # adjust names if dims are named 'latitude'/'longitude' or 'lat'/'lon'
# val = ds["so"].sel(
#     latitude=lat,
#     longitude=lon,
#     method="nearest"
# ).values

# print(f"At ({lat}, {lon}), so = {val}")

/tmp/ipykernel_530513/1593599888.py:10: DtypeWarning: Columns (25,73,88,93,94,95,96,97,98,99,100,101,102,104,106,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,128,129,130,132,133,135,136,137,138,139,140,143,144,145,146,147,148,149,150,151,153,154,155,156,157,158,159,160,161,163,164,165,166,167,169,170,171,172,173,174,175,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  occurrences = pd.read_csv("data/occurrences.csv")


-32.0959727101363


ValueError: cannot use a dict-like object for selection on a dimension that does not have a MultiIndex